<a href="https://colab.research.google.com/github/gigifs/NLW/blob/master/Sumarizacao_Noticias_PTBR_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Sumarização Automática de Notícias em Português

**Integrantes:** [Nomes]

**Disciplina:** Ciência de Dados e Aprendizagem de Máquina

**Data:** 14 de Outubro de 2025

## 🎯 Objetivo

Este notebook demonstra um sistema completo para sumarizar notícias a partir de links da web. Abordamos e comparamos duas técnicas principais de sumarização:

1.  **Extrativa (LexRank):** Seleciona as frases mais importantes do texto original.
2.  **Abstrativa (Modelo T5):** Gera um novo texto, reescrevendo as ideias principais com "palavras próprias".

Este material serve como a demonstração prática do nosso seminário e como um tutorial para quem desejar replicar o projeto.

## ⚙️ 1. Configuração Completa do Ambiente

O primeiro passo em qualquer projeto de programação é garantir que temos todas as "peças" e "ferramentas" necessárias. A célula de código a seguir irá preparar nosso ambiente de trabalho, realizando duas tarefas essenciais:

1.  **Instalação de Bibliotecas Python:** São pacotes de código que nos dão superpoderes, adicionando funcionalidades prontas para uso.
2.  **Download de Componentes Adicionais:** Algumas bibliotecas precisam de recursos extras para funcionar, como dicionários ou modelos de tokenização.


### Principais Ferramentas que Estamos Instalando:

* **`trafilatura`**: Para extrair o texto principal de páginas da web.
* **`sumy`**: Para a sumarização extrativa (método LexRank).
* **`transformers`**: A biblioteca do Hugging Face para usar o modelo T5 (sumarização abstrativa).
* **`rouge_score`**: Para avaliar a qualidade dos nossos resumos.
* **`nltk`**: Uma biblioteca clássica de processamento de linguagem natural.

---
***(Execute a célula de código a seguir apenas uma vez no início da sessão)***

In [ ]:
# =================================================================
# BLOCO 1: INSTALAÇÃO DE DEPENDÊNCIAS E CONFIGURAÇÃO
# =================================================================

# 1. Instalação das bibliotecas Python principais via pip
#    Usamos a flag "-q" em alguns para uma instalação mais silenciosa.
!pip -q install trafilatura==1.9.0 sumy==0.11.0 transformers==4.44.2 sentencepiece==0.2.0 accelerate==0.34.2 torch --upgrade
!pip -q install evaluate==0.4.2 datasets==2.21.0 langdetect==1.0.9
!pip install rouge_score
# Obs.: O PyTorch, essencial para o modelo T5, é instalado na primeira linha.

# 2. Download do componente 'punkt' para a biblioteca NLTK
#    Este recurso é um tokenizador que ensina o 'sumy' a separar frases corretamente.
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

# 3. Mensagem de confirmação para o usuário
print("\n✅ Ambiente configurado com sucesso! Todas as dependências foram instaladas.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/2

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.



✅ Ambiente configurado com sucesso! Todas as dependências foram instaladas.


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## 📚 2. Importação das Bibliotecas (Nossa "Mesa de Trabalho")

Agora que nosso ambiente está configurado, o próximo passo é "colocar as ferramentas na mesa". Em Python, fazemos isso através do comando `import`.

Pense nisso como um chef de cozinha que, antes de começar a preparar um prato, organiza todos os ingredientes e utensílios que vai precisar. A célula de código a seguir importa todas as bibliotecas que definimos na etapa anterior, tornando suas funcionalidades prontas para serem usadas no nosso projeto.

Cada grupo de `import` representa uma categoria de ferramentas para:
* Manipulação de texto e listas.
* Extração de conteúdo da web (`trafilatura`).
* Sumarização Extrativa (`sumy`).
* Sumarização Abstrativa (`transformers`).
* Garantir a reprodutibilidade dos resultados.

In [ ]:
# =============================================================
# 2) Importações e utilitários
# =============================================================

# --- Ferramentas gerais para manipulação de texto e sistema ---
import re
import math
import textwrap
from typing import Optional, List

# --- Nossa ferramenta para extrair conteúdo de notícias ---
import trafilatura

# --- Para a sumarização EXTRATIVA (LexRank) ---
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

# --- Para a sumarização ABSTRATIVA (Transformers) ---
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# --- Bibliotecas de avaliação e utilidades ---
import evaluate
import numpy as np

# --- Garantindo a Reprodutibilidade ---
# Fixamos a "semente" aleatória (seed) para que, sempre que o código for
# executado, os resultados que dependem de alguma aleatoriedade sejam
# os mesmos. Isso é uma boa prática em ciência de dados.
import random
random.seed(42)
np.random.seed(42)
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 42

# Mensagem de confirmação
print("✅ Pacotes importados com sucesso. Ferramentas prontas para uso!")

✅ Pacotes importados com sucesso. Ferramentas prontas para uso!


## 🛠️ 3. As Ferramentas: Funções do Nosso Pipeline

Com as bibliotecas importadas, agora vamos definir as funções principais que formarão a base do nosso sistema. Cada função tem uma responsabilidade clara no processo de transformar uma página da web em um texto limpo e pronto para ser resumido.

* **`fetch_article_text`**: Esta é a nossa função "garimpeira". Ela recebe uma URL, baixa o conteúdo da página e, usando a mágica do `trafilatura`, extrai de forma inteligente apenas o texto principal da notícia. É ela que remove os anúncios, menus e outros ruídos, entregando para nós somente o que importa.

* **`ensure_portuguese`**: Uma função de "controle de qualidade". Como nossos modelos de sumarização são específicos para o português, esta função dá uma rápida verificada no texto e nos avisa se o idioma detectado for outro.

* **`count_words`**: Uma função utilitária simples para contar o número de palavras em um texto, o que nos ajuda a ter uma ideia do tamanho do conteúdo original.

In [ ]:
# =============================================================
# 3) Extração e limpeza de texto de uma URL
# =============================================================

def fetch_article_text(url: str, include_comments: bool = False) -> Optional[str]:
    """
    Baixa e extrai o texto principal de um artigo de notícia usando 'trafilatura'.
    Retorna o texto limpo (string) ou None se não conseguir extrair.
    """
    try:
        downloaded = trafilatura.fetch_url(url)
        if not downloaded:
            print("⚠️ Não foi possível baixar o conteúdo da URL.")
            return None

        # 'favor_recall=True' tenta extrair o máximo de texto possível.
        extracted = trafilatura.extract(downloaded, include_comments=include_comments, favor_recall=True)
        if not extracted:
            print("⚠️ Não foi possível extrair o texto do HTML.")
            return None

        # Limpeza leve: remover múltiplas quebras de linha
        extracted = re.sub(r"\n{2,}", "\n\n", extracted).strip()
        return extracted
    except Exception as e:
        print(f"Erro ao extrair o texto: {e}")
        return None


def ensure_portuguese(text: str) -> bool:
    """
    Usa langdetect para apontar se o texto parece estar em Português.
    Apenas para alerta ao usuário (não bloqueia o fluxo).
    """
    try:
        lang = detect(text[:2000])  # detectar com amostra (se o texto for muito grande)
        if lang != 'pt':
            print(f"⚠️ Aviso: idioma detectado = '{lang}'. O texto pode não estar em PT-BR.")
            return False
        return True
    except Exception as e:
        print(f"Não foi possível detectar idioma (prosseguindo mesmo assim). Detalhe: {e}")
        return True  # não bloqueamos


def count_words(text: str) -> int:
    """
    Conta as palavras em uma string de texto.
    """
    return len(re.findall(r"\w+", text, flags=re.UNICODE))

# Mensagem de confirmação
print("✅ Funções de extração e utilitários definidas com sucesso!")

✅ Funções de extração e utilitários definidas com sucesso!


## ⛏️ 4. Ferramenta de Sumarização Extrativa (O Método "Marcador de Texto")

Agora vamos definir a primeira das nossas duas técnicas de sumarização. O método **extrativo** é o mais direto: ele lê todo o texto e seleciona as frases que considera mais importantes, sem alterá-las.

**Como funciona o `LexRank`?**
Pense em cada frase do texto como uma pessoa em uma rede social. O algoritmo `LexRank` analisa o quão similar cada frase é de todas as outras. Uma frase que se parece com muitas outras frases importantes (ou seja, que compartilha muitos conceitos) ganha "pontos de relevância". No final, o algoritmo simplesmente seleciona as frases com a maior pontuação.

A função a seguir implementa exatamente isso: ela recebe um texto e nos devolve um número pré-definido das frases mais "centrais" daquele texto.

In [ ]:
# =============================================================
# 4) Sumarização EXTRATIVA (LexRank via sumy)
# =============================================================

def summarize_extractive_lexrank(text: str, num_sentences: int = 3) -> str:
    """
    Aplica LexRank (sumy) para selecionar as frases mais importantes.
    - num_sentences: quantas frases queremos no resumo.
    """
    # Passo 1: Preparamos o texto para que a biblioteca 'sumy' possa entendê-lo.
    # Usamos um 'Tokenizer' para o idioma português.
    parser = PlaintextParser.from_string(text, Tokenizer("portuguese"))

    # Passo 2: Inicializamos o sumarizador LexRank.
    summarizer = LexRankSummarizer()

    # Passo 3: Aqui a mágica acontece. O sumarizador "lê" o documento
    # e seleciona o número de sentenças mais relevantes.
    sentences = summarizer(parser.document, sentences_count=num_sentences)

    # Passo 4: Juntamos as sentenças escolhidas para formar o resumo final.
    summary = " ".join(str(s) for s in sentences)
    return summary.strip()

# Mensagem de confirmação
print("✅ Função 'summarize_extractive_lexrank' definida com sucesso!")

✅ Função 'summarize_extractive_lexrank' definida com sucesso!


## 🧠 5. Sumarização Abstrativa (O Método "Inteligente")

Diferente do método extrativo, a abordagem **abstrativa** não copia frases. Ela busca **compreender** o texto e **reescrevê-lo** com novas palavras, de forma concisa e coerente — exatamente como um ser humano faria.

Para isso, usaremos um "cérebro digital": um modelo de linguagem pré-treinado da família **T5 (Text-to-Text Transfer Transformer)**, que foi ajustado especificamente para resumir textos em português.

A célula a seguir irá baixar este modelo (que pode ter milhões de parâmetros) e carregá-lo na memória. **Este passo pode demorar de 1 a 2 minutos**, dependendo da conexão.

In [ ]:
# =============================================================
# 5.1) Carregando o Modelo T5 e o Tokenizador
# =============================================================

# Definimos o nome do modelo que vamos baixar do repositório Hugging Face.
# Este é um modelo popular e estável para sumarização em português.
MODEL_NAME = "recogna-nlp/ptt5-base-summ"

# O Tokenizador é como um "tradutor": ele converte nosso texto em números
# que são o único idioma que o modelo de IA entende.
print("Baixando o tokenizador...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Agora, carregamos o modelo em si. Esta é a parte que pode demorar.
print(f"Baixando e carregando o modelo '{MODEL_NAME}'...")
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

print("\n✅ Modelo T5 carregado com sucesso! O 'cérebro' está pronto.")

Baixando o tokenizador...


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/756k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Baixando e carregando o modelo 'recogna-nlp/ptt5-base-summ'...


config.json:   0%|          | 0.00/780 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]


✅ Modelo T5 carregado com sucesso! O 'cérebro' está pronto.


### 5.2. A Lógica da Sumarização Abstrativa

Com o modelo carregado, precisamos de funções que saibam como "conversar" com ele.

* **`chunk_text`**: Modelos de IA possuem um limite de "memória de curto prazo" (o tamanho do texto que conseguem analisar de uma vez). Esta função auxiliar serve como uma "tesoura" inteligente, quebrando textos muito longos em pedaços menores que o T5 consegue processar sem se perder.
* **`summarize_abstractive_t5`**: Esta é a função principal. Ela pega o texto (já dividido em pedaços, se necessário), adiciona a instrução `"sumarize: "`, envia para o modelo T5 e depois traduz a resposta numérica de volta para um texto legível para nós.

In [ ]:
# =============================================================
# 5.2) Definição das Funções para Sumarização Abstrativa
# =============================================================

def chunk_text(text: str, max_tokens: int = 512) -> List[str]:
    """
    Divide textos longos em blocos menores respeitando o limite de tokens do modelo.
    O T5 trabalha melhor com entradas curtas.
    """
    sentences = re.split(r"(?<=[\.!?])\s+", text)
    chunks = []
    current = ""
    for sent in sentences:
        test = (current + " " + sent).strip()
        if len(tokenizer.encode(test, truncation=False)) <= max_tokens:
            current = test
        else:
            if current:
                chunks.append(current)
            current = sent
    if current:
        chunks.append(current)
    return chunks

def summarize_abstractive_t5(text: str, max_new_tokens: int = 120, do_sample: bool = False) -> str:
    """
    Gera um resumo abstrativo com T5 em PT-BR.
    - max_new_tokens: controla o tamanho máximo do resumo.
    - do_sample=False (determinístico); True pode deixar mais criativo (use com cuidado).
    """
    # O modelo foi treinado para entender esta instrução específica.
    prefix = "sumarize: "

    # Primeiro, quebramos o texto em pedaços, se necessário.
    chunks = chunk_text(text)
    summaries = []

    # Geramos um resumo para cada pedaço.
    for ch in chunks:
        input_ids = tokenizer.encode(prefix + ch, return_tensors="pt", truncation=True, max_length=512)
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            num_beams=4,              # Técnica para buscar resultados de melhor qualidade.
            do_sample=do_sample,
            early_stopping=True,      # Para a geração quando a frase está completa.
            no_repeat_ngram_size=3    # Evita repetições de palavras no resumo.
        )
        chunk_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        summaries.append(chunk_summary.strip())

    # Se o texto foi quebrado, juntamos os resumos parciais e fazemos um
    # "resumo do resumo" para um resultado final mais coeso.
    combined = " ".join(summaries)
    if len(chunks) > 1:
        input_ids = tokenizer.encode(prefix + combined, return_tensors="pt", truncation=True, max_length=512)
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens, # Usamos o tamanho original para o resumo final
            num_beams=4,
            do_sample=False,
            early_stopping=True,
            no_repeat_ngram_size=3
        )
        combined = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    return combined

# Mensagem de confirmação
print("✅ Funções 'chunk_text' e 'summarize_abstractive_t5' definidas com sucesso!")

✅ Funções 'chunk_text' e 'summarize_abstractive_t5' definidas com sucesso!


## 📏 6. Ferramentas de Avaliação (A "Régua" de Qualidade)

Com as funções de sumarização prontas, como podemos saber se um resumo é "bom"? Nesta etapa, definimos duas funções para nos ajudar a medir a qualidade e a eficiência dos resultados.

* **`compute_rouge_manual`**: Esta é a nossa principal ferramenta de avaliação de qualidade. A métrica **ROUGE** funciona comparando o resumo gerado pelo nosso programa com um resumo de referência "perfeito" (geralmente feito por um humano). Ela mede a sobreposição de palavras e sequências entre os dois textos.
    * **ROUGE-1**: Mede a sobreposição de palavras individuais.
    * **ROUGE-2**: Mede a sobreposição de pares de palavras (bigramas).
    * **ROUGE-L**: Mede a sequência de palavras em comum mais longa, avaliando a estrutura da frase.

* **`compression_ratio`** (Fator de Compressão): Uma métrica simples que nos diz o quão "agressivo" foi o resumo. Ela calcula a porcentagem do tamanho do resumo em relação ao texto original. Um número menor significa um resumo mais curto e uma compressão maior.

In [ ]:
# =============================================================
# 6) Funções de Avaliação
# =============================================================
# Observação: ROUGE compara o resumo gerado com um resumo de referência (humano).

from rouge_score import rouge_scorer

def compute_rouge_manual(prediction: str, reference: str):
    """
    Calcula ROUGE-1, ROUGE-2 e ROUGE-L entre o resumo gerado (prediction)
    e um resumo de referência (reference).
    Retorna um dicionário com os escores F1-Score de cada métrica.
    """
    # Inicializa o 'scorer' com as métricas que queremos calcular.
    # 'use_stemmer=True' ajuda a comparar palavras com a mesma raiz (ex: "correr" e "correndo").
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Calcula a pontuação
    scores = scorer.score(reference, prediction)

    # Retornamos apenas o F1-Score de cada métrica para uma visualização mais simples.
    return {k: v.fmeasure for k, v in scores.items()}

def compression_ratio(original_text: str, summary_text: str) -> float:
    """
    Retorna o fator de compressão: (#palavras_resumo / #palavras_original).
    Quanto menor, maior a compressão.
    """
    # Função interna para contar as palavras
    def _count_words(t: str) -> int:
        return len(re.findall(r"\w+", t, flags=re.UNICODE))

    # Conta as palavras no texto original e no resumo
    original_words = _count_words(original_text)
    summary_words = _count_words(summary_text)

    # Calcula a razão para obter o fator de compressão
    return (summary_words / original_words) if original_words > 0 else 0.0

# Mensagem de confirmação
print("✅ Funções de avaliação 'compute_rouge_manual' e 'compression_ratio' definidas!")

✅ Funções de avaliação 'compute_rouge_manual' e 'compression_ratio' definidas!


## 🎬 7. A Grande Demonstração: Juntando Todas as Peças

Agora que todas as nossas ferramentas estão definidas, vamos executar o pipeline completo, passo a passo, com uma notícia real.

### ▶️ Passo 1: Definir a URL da Notícia

Tudo começa com o link. Nesta célula, definimos qual notícia queremos resumir.

**(Dica para a apresentação: Tenha uma URL já testada e que não tenha bloqueios/paywall. Você pode até pegar uma notícia do dia para mostrar que o sistema funciona em tempo real!)**

In [ ]:
# =============================================================
# PASSO 1: Escolha da Notícia
# =============================================================
# Cole aqui o link da notícia para a demonstração.
url = "https://g1.globo.com/politica/noticia/2025/09/02/julgamento-de-bolsonaro-dia-1-moraes-diz-que-stf-nao-cedera-a-pressoes-gonet-traz-provas-da-trama-golpista-e-defesas-se-manifestam.ghtml"

print(f"📰 Notícia selecionada para sumarização: {url}")

📰 Notícia selecionada para sumarização: https://g1.globo.com/politica/noticia/2025/09/02/julgamento-de-bolsonaro-dia-1-moraes-diz-que-stf-nao-cedera-a-pressoes-gonet-traz-provas-da-trama-golpista-e-defesas-se-manifestam.ghtml


### ▶️ Passo 2: Extrair o Texto Limpo da Página

Agora, usamos nossa função `fetch_article_text` para "garimpar" o texto principal da notícia, removendo todo o ruído da página.

In [ ]:
# =============================================================
# PASSO 2: Extração e Verificação do Texto
# =============================================================
article_text = fetch_article_text(url)

if article_text:
    word_count = count_words(article_text)
    print(f"✅ Texto extraído com sucesso! Contém {word_count} palavras.")

    # Faz uma verificação rápida do idioma para garantir que é português
    ensure_portuguese(article_text)
else:
    print("❌ Falha ao extrair a notícia. Tente outra URL.")


✅ Texto extraído com sucesso! Contém 1752 palavras.


### ▶️ Passo 3: Criar os Resumos Extrativo e Abstrativo

Com o texto em mãos, alimentamos nossas duas funções de sumarização. A extrativa será quase instantânea, enquanto a abstrativa (que usa o modelo T5) pode levar alguns segundos para "pensar".

In [ ]:
# =============================================================
# PASSO 3: Geração dos Resumos
# =============================================================
if article_text:
    # Gerando o resumo EXTRATIVO (rápido)
    print("⛏️ Gerando resumo extrativo com LexRank...")
    extr_sum = summarize_extractive_lexrank(article_text, num_sentences=4)

    # Gerando o resumo ABSTRATIVO (pode demorar um pouco)
    print("🧠 Gerando resumo abstrativo com T5...")
    abs_sum = summarize_abstractive_t5(article_text, max_new_tokens=140)

    print("\n✅ Resumos gerados com sucesso!")

⛏️ Gerando resumo extrativo com LexRank...
🧠 Gerando resumo abstrativo com T5...

✅ Resumos gerados com sucesso!


### ▶️ Passo 4: Apresentar e Comparar os Resultados

Este é o momento principal! Vamos imprimir os dois resumos lado a lado para comparar as abordagens.

In [ ]:
# =============================================================
# PASSO 4: Comparação Visual dos Resultados
# =============================================================
if article_text:
    print("="*25, "TEXTO ORIGINAL (Trecho)", "="*25)
    print(textwrap.shorten(article_text, width=800, placeholder="..."))

    print("\n\n", "="*20, "RESUMO EXTRATIVO (LexRank)", "="*20)
    print(extr_sum)

    print("\n\n", "="*20, "RESUMO ABSTRATIVO (T5)", "="*20)
    print(abs_sum)

========================= TEXTO ORIGINAL (Trecho) =========================
O primeiro dia do julgamento de Bolsonaro e outros sete réus pela trama golpista teve recado do ministro do Supremo Tribunal Federal (STF) Alexandre de Moraes (relator do processo), acusação do procurador-geral da República, Paulo Gonet, e defesa dos acusados. A sessão será retomada nesta quarta-feira (3), às 9h, com as defesas do ex-presidente e de três generais. 🔴 O g1 vai transmitir ao vivo o julgamento do núcleo crucial da trama golpista. ACOMPANHE AQUI Moraes, o primeiro a falar, citou pressões internas e externas sobre a Corte, mas ressaltou que o papel do Supremo é julgar com imparcialidade e independência. - Gonet listou as provas da trama golpista, disse que não é necessária uma assinatura para se configurar o crime de golpe de Estado e pediu a condenação de todo o núcleo...


 ==================== RESUMO EXTRATIVO (LexRank) ====================
O primeiro dia do julgamento de Bolsonaro e outros sete r

### ▶️ Passo 5: Análise Quantitativa

Finalmente, vamos usar nossas funções de avaliação para analisar os resumos numericamente, verificando o fator de compressão de cada um.

In [ ]:
# =============================================================
# PASSO 5: Métricas de Compressão
# =============================================================
if article_text:
    print("="*25, "Métricas de Compressão", "="*25)

    print(f"Original: {count_words(article_text)} palavras")
    print(f"Extrativo: {count_words(extr_sum)} palavras (Fator de Compressão: {compression_ratio(article_text, extr_sum):.3f})")
    print(f"Abstrativo: {count_words(abs_sum)} palavras (Fator de Compressão: {compression_ratio(article_text, abs_sum):.3f})")

========================= Métricas de Compressão =========================
Original: 1752 palavras
Extrativo: 171 palavras (Fator de Compressão: 0.098)
Abstrativo: 44 palavras (Fator de Compressão: 0.025)


## ▶️ Passo 6 (Opcional, mas Recomendado): Avaliação com ROUGE

Finalmente, para ir além da análise visual, podemos usar a métrica ROUGE para avaliar numericamente nossos resumos. Para isso, precisamos de um **resumo de referência**, ou seja, um resumo "ideal" escrito por um humano para servir de gabarito.

A célula abaixo calcula a pontuação ROUGE para os dois resumos que geramos, comparando-os com a referência que definimos.

In [ ]:
# ====================================================================
# PASSO 6: Avaliação Quantitativa com ROUGE
# ====================================================================

# Cole aqui um resumo de referência para a notícia que você está analisando.
# Este resumo foi criado para a notícia sobre a venda de atestados falsos.
referencia_humana = "Sessão abriu julgamento do núcleo central da trama golpista no Supremo. Réus respondem por crimes como tentativa de golpe de Estado e organização criminosa armada. Advogados contestam acusações.".strip()

if 'article_text' in globals() and article_text and referencia_humana and "Insira aqui" not in referencia_humana:
    print(f"Usando a seguinte referência humana:\n'{referencia_humana}'")

    print("\nCalculando ROUGE para o resumo EXTRATIVO...")
    rouge_extr = compute_rouge_manual(extr_sum, referencia_humana)
    print(rouge_extr)

    print("\nCalculando ROUGE para o resumo ABSTRATIVO...")
    rouge_abs = compute_rouge_manual(abs_sum, referencia_humana)
    print(rouge_abs)
else:
    print("⚠️ Para calcular o ROUGE, primeiro execute os passos anteriores e defina a variável 'referencia_humana'.")

Usando a seguinte referência humana:
'Sessão abriu julgamento do núcleo central da trama golpista no Supremo. Réus respondem por crimes como tentativa de golpe de Estado e organização criminosa armada. Advogados contestam acusações.'

Calculando ROUGE para o resumo EXTRATIVO...
{'rouge1': 0.19555555555555554, 'rouge2': 0.07174887892376683, 'rougeL': 0.1333333333333333}

Calculando ROUGE para o resumo ABSTRATIVO...
{'rouge1': 0.29268292682926833, 'rouge2': 0.05, 'rougeL': 0.12195121951219513}


## ✅ Conclusão e Próximos Passos

Este projeto demonstrou com sucesso a construção de um pipeline completo e funcional de sumarização de notícias em português, desde a extração do conteúdo de uma URL até a geração de resumos por meio de duas abordagens distintas.

### Principais Conclusões

* **Aplicabilidade Real:** O pipeline `URL → Texto → Resumo` provou ser uma solução prática e com utilidade imediata para lidar com o excesso de informação online.
* **Riqueza da Comparação:** A análise lado a lado dos métodos **extrativo (LexRank)** e **abstrativo (T5)** não apenas gerou resultados diferentes, mas também enriqueceu a compreensão teórica sobre as forças e fraquezas de cada abordagem.

### Sugestões para o Futuro

O trabalho desenvolvido aqui serve como uma excelente base para projetos ainda mais ambiciosos. Algumas possibilidades incluem:

* **Ajuste Fino (*Fine-tuning*) do Modelo:** Para obter resumos ainda mais precisos e adaptados a um nicho específico (ex: notícias de finanças, artigos científicos), o modelo T5 poderia ser treinado em um conjunto de dados especializado.
* **Criação de uma Interface Web:** Utilizando ferramentas simples como **Gradio** ou **Streamlit**, seria possível transformar este notebook em uma aplicação web amigável, onde qualquer pessoa poderia colar uma URL e receber o resumo, sem precisar interagir com o código.